In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [ ]:
df=pd.read_csv("/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
sns.set_theme()
df.hist(bins=50,figsize=(20,15))
plt.show()

In [ ]:
df_copy=df.copy()

In [ ]:
scaler=preprocessing.StandardScaler()
standardized=scaler.fit_transform(df_copy)

In [ ]:
df_stand=pd.DataFrame(data=standardized,columns=["age","anaemia","creatinine_phosphokinase","diabetes","ejection_fraction","high_blood_pressure",
                                                "platelets","serum_creatinine","serum_sodium","sex","smoking","time","DEATH_EVENT"]) 

In [ ]:
df_stand.describe()

In [ ]:
df_stand.head()

In [ ]:
target=df.DEATH_EVENT
df_stand.insert((df_stand.shape[1]),"DEATH",target)

In [ ]:
df_stand.drop(columns={'DEATH_EVENT'}, axis=1, inplace=True)
df_stand.drop(columns={'time'}, axis=1, inplace=True)

In [ ]:
df_stand.head()

In [ ]:
#split set
x=df_stand[["age","anaemia","creatinine_phosphokinase","diabetes","ejection_fraction","high_blood_pressure",
                                                "platelets","serum_creatinine","serum_sodium","sex","smoking"]]
y=df_stand.DEATH

In [ ]:
#spliting model 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_regression= LogisticRegression(random_state=0,solver="sag")

In [ ]:
#train model and make predictions
log_regression_model= log_regression.fit(X_train,y_train)
log_regression_predict=log_regression_model.predict(X_test)

In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix
c_matrix=confusion_matrix(y_test,log_regression_predict)

In [ ]:
# Create pandas dataframe
dataframe = pd.DataFrame(c_matrix)
# Create heatmap
sns.heatmap(dataframe, annot=True, cbar=None, cmap="Blues")
plt.title("Confusion Matrix"), plt.tight_layout()
plt.ylabel("True Class"), plt.xlabel("Predicted Class")
plt.show()

In [ ]:
from sklearn.metrics import classification_report
# Create a classification report
print(classification_report(y_test,log_regression_predict))

In [ ]:
#most impotant feature
importance=log_regression_model.coef_[0]
for col in range(len(x.columns)):
    print(x.columns[col]+":",importance[col])

sns.set_theme()
#plot feature importance
plt.figure(figsize=(15,10));
sns.barplot(["age","anaemia","creatinine_phosphokinase","diabetes","ejection_fraction","high_blood_pressure",
                                                "platelets","serum_creatinine","serum_sodium","sex","smoking"],importance);
#plt.setp(get_xticklabels(), rotation=90)
plt.xticks(rotation = 'vertical');

**LOGISTIC REGRESSION SUMMARY**

* From the feature importance graph we can see that  age, anaemia, creatinine_phosphokinase,diabetes,high blood pressure,serum creatinine and smoking plays an impotant role as a cause of death. 
* Ejection fraction,sex,serum sodium and platelets is not a factor that cause death. 
* Model has an accuracy of 72% 

**SUPPORT VECTOR MACHINE**

In [ ]:
from sklearn.svm import SVC
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#CREATE GAUSSIAN RBF KERNEL
rbf_model=SVC(kernel='rbf',probability=True)

#Creating a distribution for gamma 
gamma= uniform(loc=0, scale=10)

#Creating a distribution for gamma 
C= uniform(loc=0, scale=10)

#creating hyperparameter 
hyperparameters= dict(gamma=gamma,C=C)

In [ ]:
#create randomized search
randomizedsearch = RandomizedSearchCV(rbf_model, hyperparameters, random_state=0, n_iter=100, cv=10, verbose=0,n_jobs=-1)

In [ ]:
# Train best model using randomized search and predict. 
best_model = randomizedsearch.fit(X_train,y_train)
RBF_predict=best_model.predict(X_test)

In [ ]:
#view predictive probalbility
best_model.predict_proba(X_test)

In [ ]:
print('Best gamma:', best_model.best_estimator_.get_params()['gamma'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

In [ ]:
randomizedsearch.best_estimator_


In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix
c_matrix_for_RGB=confusion_matrix(y_test,RBF_predict)

In [ ]:
# Create pandas dataframe
dataframe = pd.DataFrame(c_matrix_for_RGB)
# Create heatmap
sns.heatmap(dataframe, annot=True, cbar=None, cmap="Blues")
plt.title("Confusion Matrix for RGB"), plt.tight_layout()
plt.ylabel("True Class"), plt.xlabel("Predicted Class")
plt.show()

In [ ]:
from sklearn.metrics import classification_report
# Create a classification report
print(classification_report(y_test,RBF_predict))

Accuracy of 69%